# Teksto klasifikavimo užduotis

Šiame modulyje pradėsime nuo paprastos teksto klasifikavimo užduoties, remdamiesi **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** duomenų rinkiniu: klasifikuosime naujienų antraštes į vieną iš 4 kategorijų: Pasaulis, Sportas, Verslas ir Mokslas/Technologijos.

## Duomenų rinkinys

Norėdami įkelti duomenų rinkinį, naudosime **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Dabar galime pasiekti mokymo ir testavimo duomenų rinkinio dalis naudodami `dataset['train']` ir `dataset['test']` atitinkamai:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Išspausdinkime pirmąsias 10 naujų antraščių iš mūsų duomenų rinkinio:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Teksto vektorizacija

Dabar turime konvertuoti tekstą į **skaičius**, kurie gali būti pateikti kaip tensoriai. Jei norime žodžių lygmens reprezentacijos, turime atlikti du dalykus:

* Naudoti **tokenizatorių**, kad tekstas būtų padalintas į **tokenus**.
* Sukurti tų tokenų **žodyną**.

### Žodyno dydžio ribojimas

AG News duomenų rinkinio pavyzdyje žodyno dydis yra gana didelis – daugiau nei 100 tūkst. žodžių. Apskritai, mums nereikia žodžių, kurie tekste pasitaiko retai — tik keliose sakiniuose jie bus, o modelis iš jų nesimokys. Todėl logiška apriboti žodyno dydį iki mažesnio skaičiaus, perduodant argumentą vektorizatoriaus konstruktoriui:

Abu šiuos veiksmus galima atlikti naudojant **TextVectorization** sluoksnį. Sukurkime vektorizatoriaus objektą ir tada iškvieskime `adapt` metodą, kad pereitume per visą tekstą ir sukurtume žodyną:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Pastaba**: mes naudojame tik dalį viso duomenų rinkinio, kad sukurtume žodyną. Tai darome tam, kad pagreitintume vykdymo laiką ir nereikėtų jūsų laukti. Tačiau prisiimame riziką, kad kai kurie žodžiai iš viso duomenų rinkinio nebus įtraukti į žodyną ir bus ignoruojami mokymo metu. Taigi, naudojant visą žodyno dydį ir apdorojant visą duomenų rinkinį per `adapt`, galutinis tikslumas turėtų padidėti, bet ne žymiai.

Dabar galime pasiekti tikrąjį žodyną:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Naudodami vektorizatorių, galime lengvai užkoduoti bet kokį tekstą į skaičių rinkinį:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Žodžių maišo (Bag-of-words) teksto reprezentacija

Kadangi žodžiai perteikia prasmę, kartais galime suprasti teksto reikšmę tiesiog pažvelgę į atskirus žodžius, nepaisant jų tvarkos sakinyje. Pavyzdžiui, klasifikuojant naujienas, tokie žodžiai kaip *oras* ir *sniegas* greičiausiai nurodys į *orų prognozę*, o žodžiai kaip *akcijos* ir *doleris* bus susiję su *finansinėmis naujienomis*.

**Žodžių maišo** (BoW) vektorinė reprezentacija yra pati paprasčiausia ir lengviausiai suprantama tradicinė vektorinė reprezentacija. Kiekvienas žodis yra susietas su vektoriaus indeksu, o vektoriaus elementas nurodo, kiek kartų tam tikras žodis pasirodo konkrečiame dokumente.

![Paveikslėlis, rodantis, kaip žodžių maišo vektorinė reprezentacija saugoma atmintyje.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note**: Taip pat galite galvoti apie BoW kaip apie visų vieno žodžio vienetinės koduotės (one-hot-encoded) vektorių sumą tekste.

Žemiau pateiktas pavyzdys, kaip sugeneruoti žodžių maišo reprezentaciją naudojant Scikit Learn python biblioteką:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Mes taip pat galime naudoti aukščiau apibrėžtą Keras vektorizatorių, konvertuodami kiekvieną žodžio numerį į vieno karšto kodavimo formatą ir sudėdami visus tuos vektorius:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Pastaba**: Gali nustebinti, kad rezultatas skiriasi nuo ankstesnio pavyzdžio. Taip yra todėl, kad Keras pavyzdyje vektoriaus ilgis atitinka žodyno dydį, kuris buvo sukurtas naudojant visą AG News duomenų rinkinį, o Scikit Learn pavyzdyje žodyną sukūrėme iš pateikto teksto vietoje.


## Mokymas BoW klasifikatoriaus

Dabar, kai išmokome sukurti žodžių maišo (bag-of-words) reprezentaciją mūsų tekstui, pereikime prie klasifikatoriaus mokymo, kuris ja naudojasi. Pirmiausia, turime konvertuoti savo duomenų rinkinį į žodžių maišo reprezentaciją. Tai galima padaryti naudojant `map` funkciją tokiu būdu:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Dabar apibrėžkime paprastą klasifikatoriaus neuroninį tinklą, kuris turi vieną linijinį sluoksnį. Įvesties dydis yra `vocab_size`, o išvesties dydis atitinka klasių skaičių (4). Kadangi sprendžiame klasifikavimo užduotį, galutinė aktyvacijos funkcija yra **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Kadangi turime 4 klases, tikslumas virš 80% yra geras rezultatas.

## Klasifikatoriaus mokymas kaip vieno tinklo

Kadangi vektorizatorius taip pat yra Keras sluoksnis, galime apibrėžti tinklą, kuris jį įtraukia, ir mokyti jį nuo pradžios iki pabaigos. Tokiu būdu nereikia vektorizuoti duomenų rinkinio naudojant `map`, tiesiog galime perduoti originalų duomenų rinkinį į tinklo įvestį.

> **Pastaba**: Vis tiek reikės taikyti `map` mūsų duomenų rinkiniui, kad laukus iš žodynų (pvz., `title`, `description` ir `label`) paverstume į poras. Tačiau, kai duomenys įkeliami iš disko, galime iš karto sukurti duomenų rinkinį su reikiama struktūra.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramai, trigramai ir n-gramai

Viena iš maišo žodžių metodo apribojimų yra ta, kad kai kurie žodžiai sudaro daugiakalbius posakius. Pavyzdžiui, žodis „hot dog“ turi visiškai kitokią reikšmę nei žodžiai „hot“ ir „dog“ kitame kontekste. Jei žodžius „hot“ ir „dog“ visada atvaizduosime tais pačiais vektoriais, tai gali suklaidinti mūsų modelį.

Norint tai išspręsti, dokumentų klasifikavimo metodai dažnai naudoja **n-gramų reprezentacijas**, kur kiekvieno žodžio, dviejų žodžių ar trijų žodžių dažnis yra naudinga savybė mokant klasifikatorius. Pavyzdžiui, bigramų reprezentacijoje į žodyną pridedame visas žodžių poras, be originalių žodžių.

Žemiau pateiktas pavyzdys, kaip sukurti bigramų maišo žodžių reprezentaciją naudojant Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Pagrindinis n-gramų metodo trūkumas yra tas, kad žodyno dydis pradeda augti itin greitai. Praktikoje mums reikia derinti n-gramų reprezentaciją su dimensijų mažinimo technika, tokia kaip *embedding'ai*, apie kuriuos kalbėsime kitame skyriuje.

Norėdami naudoti n-gramų reprezentaciją mūsų **AG News** duomenų rinkinyje, turime perduoti `ngrams` parametrą mūsų `TextVectorization` konstruktoriui. Bigramų žodyno ilgis yra **žymiai didesnis**, mūsų atveju jis viršija 1,3 milijono žodžių! Todėl yra prasminga apriboti bigramų žodžius iki tam tikro pagrįsto skaičiaus.

Galėtume naudoti tą patį kodą, kaip ir aukščiau, norėdami apmokyti klasifikatorių, tačiau tai būtų labai neefektyvu atminties atžvilgiu. Kitame skyriuje apmokysime bigramų klasifikatorių naudodami embedding'us. Tuo tarpu galite eksperimentuoti su bigramų klasifikatoriaus mokymu šiame užrašų knygelėje ir pažiūrėti, ar galite pasiekti didesnį tikslumą.


## Automatinis BoW vektorių skaičiavimas

Ankstesniame pavyzdyje BoW vektorius skaičiavome rankiniu būdu, sudėdami atskirų žodžių vieno karšto kodavimo rezultatus. Tačiau naujausia TensorFlow versija leidžia automatiškai apskaičiuoti BoW vektorius, perduodant `output_mode='count` parametrą vektorizatoriaus konstruktoriui. Tai žymiai supaprastina mūsų modelio apibrėžimą ir treniravimą:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termino dažnis - atvirkštinis dokumento dažnis (TF-IDF)

BoW reprezentacijoje žodžių pasikartojimai yra vertinami naudojant tą pačią techniką, nepriklausomai nuo paties žodžio. Tačiau akivaizdu, kad dažni žodžiai, tokie kaip *a* ir *in*, yra daug mažiau svarbūs klasifikacijai nei specializuoti terminai. Daugumoje NLP užduočių kai kurie žodžiai yra reikšmingesni nei kiti.

**TF-IDF** reiškia **termino dažnis - atvirkštinis dokumento dažnis**. Tai yra maišo su žodžiais (BoW) variacija, kur vietoj dvejetainės 0/1 reikšmės, nurodančios žodžio buvimą dokumente, naudojama slankiojo kablelio reikšmė, susijusi su žodžio pasikartojimo dažniu korpuse.

Formaliau, žodžio $i$ svoris $w_{ij}$ dokumente $j$ apibrėžiamas taip:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kur
* $tf_{ij}$ yra žodžio $i$ pasikartojimų skaičius dokumente $j$, t. y. BoW reikšmė, kurią jau matėme
* $N$ yra dokumentų skaičius kolekcijoje
* $df_i$ yra dokumentų, kuriuose yra žodis $i$, skaičius visoje kolekcijoje

TF-IDF reikšmė $w_{ij}$ didėja proporcingai žodžio pasikartojimų skaičiui dokumente ir yra koreguojama pagal dokumentų skaičių korpuse, kuriuose yra tas žodis. Tai padeda atsižvelgti į tai, kad kai kurie žodžiai pasikartoja dažniau nei kiti. Pavyzdžiui, jei žodis pasirodo *kiekviename* kolekcijos dokumente, $df_i=N$, ir $w_{ij}=0$, tokie terminai būtų visiškai ignoruojami.

TF-IDF tekstų vektorizaciją galite lengvai sukurti naudodami Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Keras bibliotekoje `TextVectorization` sluoksnis gali automatiškai apskaičiuoti TF-IDF dažnius, perduodant parametrą `output_mode='tf-idf'`. Pakartokime aukščiau naudotą kodą, kad pamatytume, ar TF-IDF naudojimas padidina tikslumą:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Išvada

Nors TF-IDF reprezentacijos suteikia skirtingiems žodžiams dažnio svorius, jos nesugeba perteikti prasmės ar tvarkos. Kaip garsus lingvistas J. R. Firth pasakė 1935 m., „Visapusiška žodžio prasmė visada yra kontekstinė, ir joks prasmės tyrimas, atskirtas nuo konteksto, negali būti laikomas rimtu.“ Vėliau kurse išmoksime, kaip iš teksto išgauti kontekstinę informaciją naudojant kalbos modeliavimą.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus aiškinimus, kylančius dėl šio vertimo naudojimo.
